# 1. Readings

## 1.1. Text summarizer

Based on https://towardsdatascience.com/summarizing-tweets-in-a-disaster-part-ii-67db021d378d:
- look for situational words, describing situation or casulties using SpaCy (Numerals (eg. number of casualties, important phone numbers); Entities (eg. places, dates, events, organisations, etc.))
    - use entity-types, look for content words
- tf-idf score (rank somthing like "Nepal" highly, but not "the") --> use Textacy
- clean data before tokenizing: abbreviations, misspellings (NLTK has a twitter-specific tokenizer)
- summary of words as an ILP problem

check also the notebooks
- for SpaCy: https://github.com/gabrieltseng/datascience-projects/blob/master/natural_language_processing/twitter_disasters/spaCy/3%20-%20Abstractive%20Summary.ipynb
- for NLTK: https://github.com/gabrieltseng/datascience-projects/blob/master/natural_language_processing/twitter_disasters/NLTK/3%20-%20Abstractive%20Summary.ipynb

- ideas for overall approach: use occuring tweets as well (e.g. twitter set for wildfire)

## 1.2. Keyword Extraction

- based on https://medium.com/analytics-vidhya/automated-keyword-extraction-from-articles-using-nlp-bfd864f41b34
- also very interesting points on text pre-processing in here

<img src = "KeyWordExtraction_HighLevel.png" width="500">

## 1.x. Tools

- word embeddings: https://www.wikiwand.com/en/Word_embedding --> check word2vec
- sentiment analysis: https://www.wikiwand.com/en/Sentiment_analysis
    - for background on singular value decomposition https://www.wikiwand.com/en/Singular_value_decomposition
- using word graphs (powerful when there are multiple sentences describing similar situations)
- linguistic quality: compare my sample sentence to "normal" English sentences
    - see also KenLM tool at https://kheafield.com/code/kenlm/
    - and more readings to understand this challenge http://masatohagiwara.net/training-an-n-gram-language-model-and-estimating-sentence-probability.html
    - can be compared to current "correct" American English https://www.english-corpora.org/coca/

# 2. Disaster datasets

## 2.1. Twitter datasets

- https://arxiv.org/abs/1605.05894
- https://www.aaai.org/ocs/index.php/ICWSM/ICWSM11/paper/view/2834
- https://dl.acm.org/citation.cfm?id=2914600